In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)
indices

In [ ]:
import numpy as np  #for later code
from scipy.spatial import cKDTree
import pandas as pd

df = pd.read_csv('data/busstops_norway.csv')
liste = np.empty()
for index in range(len(df)):
    lvl1 = df._get_value(index,'lv2')
    lat1 = df._get_value(index,'lat')
    lon1 = df._get_value(index,'lon')
    liste.append(



kdtree = cKDTree(liste)
NN_dists = np.zeros(numPoints)  #pre-allocate an array to store distances
for i in range(numPoints):
    point = xyzData[i]

    NN_dist, NN_idx = kdtree.query(point,k=[1])

    #Note: 'k' specifies the kth neighbor distance to compute, 
    #so set k=2 if you end up finding the point as its own "neighbor":
    if NN_dist == 0:
        NN_dist, NN_idx = targetTree.query(curCoord,k=[2])
    
    NN_dists[i] = NN_dist


def stores_within_distance(distance):
    train_data['num_stores_within_distance'] = 0
    for index in range(len(train_data)):
        lvl1 = train_data._get_value(index,'lv2')
        lat1 = train_data._get_value(index,'lat')
        lon1 = train_data._get_value(index,'lon')
        count = 0
        for inx in range(len(train_data)):
            lvl2 = train_data._get_value(inx,'lv2')
            lat2 = train_data._get_value(inx,'lat')
            lon2 = train_data._get_value(inx,'lon')
            try:
                dist = geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km #some values have nan in them. must be fixed. 
                print(lat1, lat2, lon1, lon2)
            except:
                print(lat1, lat2, lon1, lon2)

            if dist < distance and lvl1 == lvl2:
                count += 1
        train_data._set_value(index, 'num_stores_within_distance', count- 1)